In [3]:
"""
Завдання 1
Створіть співпрограму, яка отримує контент із зазначених посилань і логує хід виконання в database, використовуючи стандартну бібліотеку requests, 
а потім проробіть те саме з бібліотекою aiohttp. Кроки, які мають бути залоговані: початок запиту до адреси X, відповідь для адреси X отримано 
зі статусом 200. Перевірте хід виконання програми на >3 ресурсах і перегляньте послідовність запису логів в обох варіантах і порівняйте результати. 
Для двох видів завдань використовуйте різні файли для логування, щоби порівняти отриманий результат. 
"""
import sqlite3
from sqlite3 import Error

def create_connection(path):
    connection = None
    try:
        connection = sqlite3.connect(path)
        print("Connection to SQLite DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query executed successfully")
    except Error as e:
        print(f"The error '{e}' occurred")
def execute_read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Error as e:
        print(f"The error '{e}' occurred")

def read_all_data(table_name):
    select_records = "SELECT * from " + table_name
    records = execute_read_query(connection, select_records)

    for rec in records:
        print(rec)

connection = create_connection("./data/sqliteDB")

create_table = """
CREATE TABLE IF NOT EXISTS logging_sync (
  id INTEGER PRIMARY KEY AUTOINCREMENT,
  url TEXT NOT NULL,
  status_code TEXT NOT NULL,
  datetime TIMESTAMP NOT NULL
);
"""
execute_query(connection, create_table)

create_table = """
CREATE TABLE IF NOT EXISTS logging_async (
  id INTEGER PRIMARY KEY AUTOINCREMENT,
  url TEXT NOT NULL,
  status_code TEXT NOT NULL,
  datetime TIMESTAMP NOT NULL
);
"""
execute_query(connection, create_table)

Connection to SQLite DB successful
Query executed successfully
Query executed successfully


In [11]:
import requests
import functools
import time


def get_duration(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        start_time = time.time()
        res = func(*args, **kwargs)
        duration = time.time() - start_time
        print(f"Duration = {duration}")
        return res
    return wrapper


def download_site(url, session, count):
    with session.get(url) as response:
        status = response.status_code
        date = response.headers['Date']
        insert_query = """
            INSERT INTO
                logging_sync (url, status_code, datetime)
            VALUES
        """ + f"""("{url}", {status}, "{date}")"""
        execute_query(connection, insert_query)
        print(f"{count} --> Read {len(response.content)} from {url}")

@get_duration
def download_all_sites(sites):
    with requests.Session() as session:
        for count, url in enumerate(sites):
            download_site(url, session, count)

sites = [
    "https://github.com/",
    "https://www.jython.org"
] * 5

delete_query = "DELETE FROM logging_sync"
execute_query(connection, delete_query)
download_all_sites(sites)

Query executed successfully
Query executed successfully
0 --> Read 225854 from https://github.com/
Query executed successfully
1 --> Read 10782 from https://www.jython.org
Query executed successfully
2 --> Read 225654 from https://github.com/
Query executed successfully
3 --> Read 10782 from https://www.jython.org
Query executed successfully
4 --> Read 225654 from https://github.com/
Query executed successfully
5 --> Read 10782 from https://www.jython.org
Query executed successfully
6 --> Read 225654 from https://github.com/
Query executed successfully
7 --> Read 10782 from https://www.jython.org
Query executed successfully
8 --> Read 225644 from https://github.com/
Query executed successfully
9 --> Read 10782 from https://www.jython.org
Duration = 1.4793338775634766


In [12]:
read_all_data('logging_sync')

(11, 'https://github.com/', '200', 'Fri, 31 Mar 2023 10:57:57 GMT')
(12, 'https://www.jython.org', '200', 'Fri, 31 Mar 2023 10:58:07 GMT')
(13, 'https://github.com/', '200', 'Fri, 31 Mar 2023 10:57:58 GMT')
(14, 'https://www.jython.org', '200', 'Fri, 31 Mar 2023 10:58:07 GMT')
(15, 'https://github.com/', '200', 'Fri, 31 Mar 2023 10:57:58 GMT')
(16, 'https://www.jython.org', '200', 'Fri, 31 Mar 2023 10:58:08 GMT')
(17, 'https://github.com/', '200', 'Fri, 31 Mar 2023 10:57:58 GMT')
(18, 'https://www.jython.org', '200', 'Fri, 31 Mar 2023 10:58:08 GMT')
(19, 'https://github.com/', '200', 'Fri, 31 Mar 2023 10:58:08 GMT')
(20, 'https://www.jython.org', '200', 'Fri, 31 Mar 2023 10:58:08 GMT')


In [51]:
import asyncio
import time
import aiohttp
import functools


async def download_site(session, url):
    async with session.get(url) as response:        
        print(f"Read {response.content_length} from {url}")
        status = response.status
        date = response.headers['Date']
        insert_query = """
            INSERT INTO
                logging_async (url, status_code, datetime)
            VALUES
        """ + f"""("{url}", {status}, "{date}")"""
        execute_query(connection, insert_query)
        


async def download_all_sites(sites):
    async with aiohttp.ClientSession() as session:
        tasks = []
        for url in sites:
            task = asyncio.ensure_future(download_site(session, url))
            tasks.append(task)
        await asyncio.gather(*tasks, return_exceptions=True)

sites = [
    "https://ipython.org/",
    "https://www.jython.org"
] * 5

if __name__ == "__main__":
    delete_query = "DELETE FROM logging_async"
    #execute_query(connection, delete_query)
    asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())
    start_time = time.time()
    await download_all_sites(sites)
    duration = time.time() - start_time
    print(f"Duration = {duration}")

Read 3721 from https://www.jython.org
Query executed successfully
Read 3721 from https://www.jython.org
Query executed successfully
Read 3721 from https://www.jython.org
Query executed successfully
Read 3721 from https://www.jython.org
Query executed successfully
Read 3721 from https://www.jython.org
Query executed successfully
Read 5507 from https://ipython.org/
Query executed successfully
Read 5507 from https://ipython.org/
Query executed successfully
Read 5507 from https://ipython.org/
Query executed successfully
Read 5507 from https://ipython.org/
Query executed successfully
Read 5507 from https://ipython.org/
Query executed successfully
Duration = 0.36815762519836426


In [52]:
read_all_data('logging_async')

(1, 'https://www.jython.org', '200', 'Fri, 31 Mar 2023 11:26:16 GMT')
(2, 'https://www.jython.org', '200', 'Fri, 31 Mar 2023 11:26:16 GMT')
(3, 'https://www.jython.org', '200', 'Fri, 31 Mar 2023 11:26:16 GMT')
(4, 'https://www.jython.org', '200', 'Fri, 31 Mar 2023 11:26:16 GMT')
(5, 'https://www.jython.org', '200', 'Fri, 31 Mar 2023 11:26:16 GMT')
(6, 'https://ipython.org/', '200', 'Fri, 31 Mar 2023 11:26:16 GMT')
(7, 'https://ipython.org/', '200', 'Fri, 31 Mar 2023 11:26:16 GMT')
(8, 'https://ipython.org/', '200', 'Fri, 31 Mar 2023 11:26:16 GMT')
(9, 'https://ipython.org/', '200', 'Fri, 31 Mar 2023 11:26:16 GMT')
(10, 'https://ipython.org/', '200', 'Fri, 31 Mar 2023 11:26:16 GMT')
